### 豆瓣评分的预测

在这个项目中，我们要预测一部电影的评分，这个问题实际上就是一个分类问题。给定的输入为一段文本，输出为具体的评分。 在这个项目中，我们需要做：
- 文本的预处理，如停用词的过滤，低频词的过滤，特殊符号的过滤等
- 文本转化成向量，将使用三种方式，分别为tf-idf, word2vec以及BERT向量。 
- 训练逻辑回归和朴素贝叶斯模型，并做交叉验证
- 评估模型的准确率

在具体标记为``TODO``的部分填写相应的代码。 

In [1]:
#导入数据处理的基础包
import numpy as np
import pandas as pd

#导入用于计数的包
from collections import Counter

#导入tf-idf相关的包
from sklearn.feature_extraction.text import TfidfTransformer    
from sklearn.feature_extraction.text import CountVectorizer

#导入模型评估的包
from sklearn import metrics

#导入与word2vec相关的包
from gensim.models import KeyedVectors

#导入与bert embedding相关的包，关于mxnet包下载的注意事项参考实验手册
from bert_embedding import BertEmbedding
import mxnet

#包tqdm是用来对可迭代对象执行时生成一个进度条用以监视程序运行过程
from tqdm import tqdm

#导入其他一些功能包
import requests
import os

### 1. 读取数据并做文本的处理
你需要完成以下几步操作：
- 去掉无用的字符如！&，可自行定义
- 中文分词
- 去掉低频词

In [2]:
#读取数据
data = pd.read_csv('data/DMSC.csv')
#观察数据格式
data.head()

,ID,Movie_Name_EN,Movie_Name_CN,Crawl_Date,Number,Username,Date,Star,Comment,Like
0,0,Avengers Age of Ultron,复仇者联盟2,2017-01-22,1,然潘,2015-05-13,3,连奥创都知道整容要去韩国。,2404
1,10,Avengers Age of Ultron,复仇者联盟2,2017-01-22,11,影志,2015-04-30,4,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,381
2,20,Avengers Age of Ultron,复仇者联盟2,2017-01-22,21,随时流感,2015-04-28,2,奥创弱爆了弱爆了弱爆了啊！！！！！！,120
3,30,Avengers Age of Ultron,复仇者联盟2,2017-01-22,31,乌鸦火堂,2015-05-08,4,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,30
4,40,Avengers Age of Ultron,复仇者联盟2,2017-01-22,41,办公室甜心,2015-05-10,5,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,16


In [3]:
#输出数据的一些相关信息
#data.info()

In [4]:
#只保留数据中我们需要的两列：Comment列和Star列
data = data[['Comment','Star']]
#观察新的数据的格式
data.head()

,Comment,Star
0,连奥创都知道整容要去韩国。,3
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,4
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,2
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,4
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,5


In [5]:
# 这里的star代表具体的评分。但在这个项目中，我们要预测的是正面还是负面。我们把评分为1和2的看作是负面，把评分为3，4，5的作为正面
data['Star']=(data.Star/3).astype(int)
data

,Comment,Star
0,连奥创都知道整容要去韩国。,1
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,1
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,1
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,1
...,...,...
212501,里里外外我都打满分，太赞了。,1
212502,超棒！！！拟人超级像的，每个配角都好有戏，每个动物都好萌，想摸摸毛，fox一直叫bunny...,1
212503,狐狸确实帅,1
212504,不负我望，超级好看！,1


#### 任务1： 去掉一些无用的字符

In [6]:
# TODO1: 去掉一些无用的字符，自行定一个字符几何，并从文本中去掉
#    your to do 
import re

# Remove punctuation
data['Comment'] = data['Comment'].apply(lambda x: re.sub(r'[，。！？￥&—\“”、@.·~]', ' ', x))

# Remove space
data['Comment'] = data['Comment'] .apply(lambda x: re.sub(r"\s+", "", x))

# add English chars flag
data['Comment'] = data['Comment'] .apply(lambda x: re.sub(r"^[A-Za-z]+$", ' 英文 ', x))

# Add num flag
data['Comment'] = data['Comment'].apply(lambda x: re.sub('[0-9]+', ' 数字 ', x))

In [7]:
data

,Comment,Star
0,连奥创都知道整容要去韩国,1
1,一个没有黑暗面的人不值得信任第二部剥去冗长的铺垫开场即高潮一直到结束会有人觉得只剩动作特技不...,1
2,奥创弱爆了弱爆了弱爆了啊,0
3,与第一集不同承上启下阴郁严肃但也不会不好看啊除非本来就不喜欢漫威电影场面更加宏大单打与团战又...,1
4,看毕我激动地对友人说等等奥创要来毁灭台北怎么办厚她拍了拍我肩膀没事反正你买了两份旅行保险惹,1
...,...,...
212501,里里外外我都打满分太赞了,1
212502,超棒拟人超级像的每个配角都好有戏每个动物都好萌想摸摸毛fox一直叫bunny胡萝卜这点超级萌...,1
212503,狐狸确实帅,1
212504,不负我望超级好看,1


#### 任务2：使用结巴分词对文本做分词

In [8]:
# TODO2: 导入中文分词包jieba, 并用jieba对原始文本做分词
import jieba
def comment_cut(content):
    seg = jieba.cut(content, cut_all=False)
    return list(seg)

# 输出进度条
tqdm.pandas(desc='apply')
data['comment_processed'] = data['Comment'].progress_apply(comment_cut)

apply:   0%|          | 0/212506 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ling\AppData\Local\Temp\jieba.cache
Loading model cost 0.762 seconds.
Prefix dict has been built successfully.
apply: 100%|██████████| 212506/212506 [00:39<00:00, 5351.59it/s]


In [9]:
# 观察新的数据的格式
data.head()

,Comment,Star,comment_processed
0,连奥创都知道整容要去韩国,1,"[连, 奥创, 都, 知道, 整容, 要, 去, 韩国]"
1,一个没有黑暗面的人不值得信任第二部剥去冗长的铺垫开场即高潮一直到结束会有人觉得只剩动作特技不...,1,"[一个, 没有, 黑暗面, 的, 人, 不, 值得, 信任, 第二部, 剥去, 冗长, 的,..."
2,奥创弱爆了弱爆了弱爆了啊,0,"[奥创, 弱, 爆, 了, 弱, 爆, 了, 弱, 爆, 了, 啊]"
3,与第一集不同承上启下阴郁严肃但也不会不好看啊除非本来就不喜欢漫威电影场面更加宏大单打与团战又...,1,"[与, 第一集, 不同, 承上启下, 阴郁, 严肃, 但, 也, 不会, 不, 好看, 啊,..."
4,看毕我激动地对友人说等等奥创要来毁灭台北怎么办厚她拍了拍我肩膀没事反正你买了两份旅行保险惹,1,"[看毕, 我, 激动, 地, 对, 友人, 说, 等等, 奥创, 要, 来, 毁灭, 台北,..."


#### 任务3：设定停用词并去掉停用词

In [10]:
# TODO3: 设定停用词并从文本中去掉停用词

# 下载中文停用词表至data/stopWord.json中，下载地址:https://github.com/goto456/stopwords/
if not os.path.exists('data/stopWord.json'):
    stopWord = requests.get("https://raw.githubusercontent.com/goto456/stopwords/master/cn_stopwords.txt")
    with open("data/stopWord.json", "wb") as f:
         f.write(stopWord.content)

# 读取下载的停用词表，并保存在列表中
with open("data/stopWord.json","r",encoding="utf8") as f:
    stopWords = f.read().split("\n")  
    
    
# 去除停用词
def rm_stop_word(wordList):
    filwords = [word for word in wordList if word not in stopWords]
    return filwords

#这行代码中.progress_apply()函数的作用等同于.apply()函数的作用，只是写成.progress_apply()函数才能被tqdm包监控从而输出进度条。
data['comment_processed'] = data['comment_processed'].progress_apply(rm_stop_word)

apply: 100%|██████████| 212506/212506 [00:44<00:00, 4765.53it/s]


In [11]:
# 观察新的数据的格式
data.head()

,Comment,Star,comment_processed
0,连奥创都知道整容要去韩国,1,"[奥创, 知道, 整容, 韩国]"
1,一个没有黑暗面的人不值得信任第二部剥去冗长的铺垫开场即高潮一直到结束会有人觉得只剩动作特技不...,1,"[一个, 没有, 黑暗面, 值得, 信任, 第二部, 剥去, 冗长, 铺垫, 开场, 高潮,..."
2,奥创弱爆了弱爆了弱爆了啊,0,"[奥创, 弱, 爆, 弱, 爆, 弱, 爆]"
3,与第一集不同承上启下阴郁严肃但也不会不好看啊除非本来就不喜欢漫威电影场面更加宏大单打与团战又...,1,"[第一集, 不同, 承上启下, 阴郁, 严肃, 不会, 好看, 本来, 喜欢, 漫威, 电影..."
4,看毕我激动地对友人说等等奥创要来毁灭台北怎么办厚她拍了拍我肩膀没事反正你买了两份旅行保险惹,1,"[看毕, 激动, 友人, 说, 奥创, 毁灭, 台北, 厚, 拍了拍, 肩膀, 没事, 反正..."


#### 任务4：去掉低频词，出现次数少于10次的词去掉

In [12]:
# TODO4: 去除低频词, 去掉词频小于10的单词，并把结果存放在data['comment_processed']里
from collections import Counter
from itertools import chain

# split words into lists
v = data['comment_processed'].tolist()

# compute global word frequency
c = Counter(chain.from_iterable(v))

# filter, join, and re-assign
data['comment_processed'] = [' '.join([j for j in i if c[j] > 9]) for i in v]


In [13]:
data

,Comment,Star,comment_processed
0,连奥创都知道整容要去韩国,1,奥创 知道 整容 韩国
1,一个没有黑暗面的人不值得信任第二部剥去冗长的铺垫开场即高潮一直到结束会有人觉得只剩动作特技不...,1,一个 没有 黑暗面 值得 信任 第二部 冗长 铺垫 开场 高潮 一直 结束 会 有人 觉得 ...
2,奥创弱爆了弱爆了弱爆了啊,0,奥创 弱 爆 弱 爆 弱 爆
3,与第一集不同承上启下阴郁严肃但也不会不好看啊除非本来就不喜欢漫威电影场面更加宏大单打与团战又...,1,第一集 不同 承上启下 阴郁 严肃 不会 好看 本来 喜欢 漫威 电影 场面 更加 宏大 团...
4,看毕我激动地对友人说等等奥创要来毁灭台北怎么办厚她拍了拍我肩膀没事反正你买了两份旅行保险惹,1,激动 友人 说 奥创 毁灭 台北 厚 肩膀 没事 反正 买 两份 旅行 惹
...,...,...,...
212501,里里外外我都打满分太赞了,1,满分 太赞
212502,超棒拟人超级像的每个配角都好有戏每个动物都好萌想摸摸毛fox一直叫bunny胡萝卜这点超级萌...,1,超棒 拟人 超级 每个 配角 有戏 每个 动物 摸摸 毛 一直 bunny 胡萝卜 这点 超...
212503,狐狸确实帅,1,狐狸 确实 帅
212504,不负我望超级好看,1,不负 超级 好看


### 2. 把文本分为训练集和测试集
选择语料库中的20%作为测试数据，剩下的作为训练数据

In [14]:
# TODO5: 把数据分为训练集和测试集. comments_train（list)保存用于训练的文本，comments_test(list)保存用于测试的文本。 y_train, y_test是对应的标签（0、1）
from sklearn.model_selection import train_test_split
test_ratio = 0.2
comments_train, comments_test, y_train, y_test = train_test_split(data['comment_processed'], data['Star'], test_size=0.2, random_state = 42, stratify=data['Star'])

In [15]:
comments_train.head()

111518                                人物 出彩 萌物 … … 剧情 其实 无聊
74565                                                    好看
61763                      大话 之后 西游 这片 百分百 贺岁 档 凑个 热闹 不失 三星
34109     土地公 可爱 适应 远 点 这种 人设 白龙 确实 从小到大 好看 白龙 特效 好看 没有 吹 神
56680                                            数字   好看 一点
Name: comment_processed, dtype: object

In [16]:
comments_test.head()

211157    画面 美 动物 萌 故事 精彩 喜欢 兔子 豹 豹 迪士尼 真的 太赞 zootopia 塑...
132545    奇 怪 完 觉得 好看 笑点 不错 剧情 效果 诚意 中二 妖魔 地方 尴尬 本来   数字...
13146                                           反正   数字   星
73506                                                徐 克星 爷
18189       有点 可爱 看不懂 梗 太 懂 梗 不萌 cp 剧情 弱 电影 一言不合 打架 一言不合 耍帅
Name: comment_processed, dtype: object

In [17]:
y_train.head()

111518    1
74565     1
61763     1
34109     1
56680     1
Name: Star, dtype: int32

In [18]:
y_test.head()

211157    1
132545    1
13146     1
73506     1
18189     1
Name: Star, dtype: int32

In [19]:
comments_train

111518                                人物 出彩 萌物 … … 剧情 其实 无聊
74565                                                    好看
61763                      大话 之后 西游 这片 百分百 贺岁 档 凑个 热闹 不失 三星
34109     土地公 可爱 适应 远 点 这种 人设 白龙 确实 从小到大 好看 白龙 特效 好看 没有 吹 神
56680                                            数字   好看 一点
                                ...                        
180862    以前 从来 不敢 丧尸 片 听说 釜山 行 好看 完 发现 丧尸 片 恐怖 反映 人性 结尾...
153171                                 前   数字   分钟 尴尬 知道 干嘛
104054        至少 流泪 片子 一个 渴望 安定 一个 渴望 自由 其实 七月 安生 一个 两面 离不开
75596                                             一般般 没有 突破
160596                                             其实 小说 不错
Name: comment_processed, Length: 170004, dtype: object

### 3. 把文本转换成向量的形式

在这个部分我们会采用三种不同的方式:
- 使用tf-idf向量
- 使用word2vec
- 使用bert向量

转换成向量之后，我们接着做模型的训练

#### 任务6：把文本转换成tf-idf向量

In [20]:
# TODO6: 把训练文本和测试文本转换成tf-idf向量。使用sklearn的feature_extraction.text.TfidfTransformer模块
#    请留意fit_transform和transform之间的区别。 常见的错误是在训练集和测试集上都使用 fit_transform，需要避免！ 
#    另外，可以留意一下结果是否为稀疏矩阵

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
transformer = TfidfTransformer()

x = vectorizer.fit_transform(comments_train)
y = vectorizer.transform(comments_test)

tfidf_train = transformer.fit_transform(x)
tfidf_test = transformer.transform(y)
print (tfidf_train.shape, tfidf_test.shape)

(170004, 14587) (42502, 14587)


#### 任务7：把文本转换成word2vec向量

In [21]:
# 由于训练出一个高效的word2vec词向量往往需要非常大的语料库与计算资源，所以我们通常不自己训练Wordvec词向量，而直接使用网上开源的已训练好的词向量。
# data/sgns.zhihu.word是从https://github.com/Embedding/Chinese-Word-Vectors下载到的预训练好的中文词向量文件
# 使用KeyedVectors.load_word2vec_format()函数加载预训练好的词向量文件
model = KeyedVectors.load_word2vec_format('data/sgns.zhihu.word')

In [22]:
#预训练词向量使用举例
model['今天']

array([-3.51068e-01,  2.57389e-01, -1.46752e-01, -4.45400e-03,
       -1.04235e-01,  3.72475e-01, -4.29349e-01, -2.80470e-02,
        1.56651e-01, -1.27600e-01, -1.68833e-01, -2.91350e-02,
        4.57850e-02, -3.53735e-01,  1.61205e-01, -1.82645e-01,
       -1.35340e-02, -2.42591e-01, -1.33356e-01, -1.31012e-01,
       -9.29500e-02, -1.70479e-01, -2.54004e-01, -1.20530e-01,
       -1.33690e-01,  7.84360e-02, -1.46603e-01, -2.77378e-01,
       -1.36723e-01,  9.29070e-02, -4.00197e-01,  2.80726e-01,
       -1.73282e-01,  8.56630e-02,  2.37251e-01,  6.24290e-02,
       -1.57132e-01,  2.15685e-01,  9.54770e-02,  1.09896e-01,
       -2.05394e-01, -3.37900e-03, -2.77480e-02,  8.16580e-02,
        9.65290e-02,  1.23188e-01,  9.55090e-02, -2.31017e-01,
       -8.59590e-02, -2.21634e-01, -1.37885e-01, -1.84790e-01,
       -2.40127e-01, -2.79150e-01, -4.56200e-03,  1.04099e-01,
        3.20523e-01, -6.77270e-02,  1.95719e-01,  4.06145e-01,
       -2.98546e-01, -1.67750e-02,  2.74917e-01, -9.023

In [23]:
# TODO7: 对于每个句子，生成句子的向量。具体的做法是：包含在句子中的所有单词的向量做平均。
vocabulary = model.vocab

# define the function to compute the mean vectors of sentences

def get_average_word2vec(context, model, generate_missing=False, k=300):
    if len(context)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [model[word] if word in model else np.random.rand(k) for word in context]
    else:
        vectorized = [model[word] if word in model else np.zeros(k) for word in context]
    return np.mean(np.array(vectorized), axis=0)

def get_word2vec_embeddings(model, tokens_comment, generate_missing=False):
    embeddings = tokens_comment.apply(lambda x: get_average_word2vec(x, model, generate_missing=generate_missing))
    return np.vstack(embeddings)

word2vec_train = get_word2vec_embeddings(model, comments_train)
word2vec_test = get_word2vec_embeddings(model, comments_test)


In [24]:
print (word2vec_train.shape, word2vec_test.shape)

(170004, 300) (42502, 300)


#### 任务8：把文本转换成bert向量

In [25]:
# 导入gpu版本的bert embedding预训练的模型。
# 若没有gpu，则ctx可使用其默认值cpu(0)。但使用cpu会使程序运行的时间变得非常慢
# 若之前没有下载过bert embedding预训练的模型，执行此句时会花费一些时间来下载预训练的模型
from bert_embedding import BertEmbedding
import mxnet
ctx = mxnet.cpu(0)
embedding = BertEmbedding(ctx=ctx)


In [26]:
# define the function to compute the mean vectors of sentences

def get_average_word2vec(context, model, k=768):
    if len(context)<1:
        return np.zeros(k)
    empty = [' ']
    vectorized = [model(word)[0][1][0] if word not in empty else np.zeros(k) for word in context ]
    
    return np.mean(np.array(vectorized), axis=0)

def get_word2vec_embeddings(model, tokens_comment, generate_missing=False):
    embeddings = tokens_comment.apply(lambda x: get_average_word2vec(x, model))
    return np.vstack(embeddings)


In [27]:
#我的码单跑一行是可以的，但是拿CPU跑太慢了，实在跑不出来

first_row_test=get_word2vec_embeddings(embedding, comments_test[0:1])
first_row_test

array([[-5.97583673e-01, -2.87807945e-01, -2.32519265e-02,
        -4.86667096e-01,  2.68476142e-01, -1.40509474e-01,
         3.71786868e-01,  1.39615455e-01, -1.67486218e-01,
        -3.39158001e-01, -4.53908113e-01, -1.93926794e-01,
         1.54324765e-01, -1.83983590e-01, -2.20380197e-02,
        -1.75222566e-01,  3.06705643e-01,  1.31488306e-01,
         4.56508186e-01,  7.87235895e-01,  8.82213977e-02,
         1.51522057e-01, -1.74331292e-01,  1.54278803e-02,
         1.25916927e-01,  6.15206012e-01, -6.31442837e-02,
        -5.71061437e-02, -2.59569860e-01,  2.53777579e-01,
         3.73142107e-01, -1.36900248e-01, -1.27385543e-01,
         2.91650135e-01, -2.30291171e-01, -3.05707358e-01,
        -1.02974146e-02, -1.44511688e-01, -3.25359475e-01,
         1.81648248e-01,  2.59573439e-01, -3.04074175e-01,
         2.15310097e-01, -3.47099310e-01, -1.75572678e-01,
        -5.65831511e-02,  2.56570424e-01,  1.17712395e-02,
        -3.49093366e-01, -1.57048126e-02, -6.16551468e-0

In [28]:
# TODO8: 跟word2vec一样，计算出训练文本和测试文本的向量，仍然采用单词向量的平均。

#bert_train=get_word2vec_embeddings(embedding, comments_train)
#bert_test=get_word2vec_embeddings(embedding, comments_test)
#print (bert_train.shape, bert_test.shape)

In [29]:
print (tfidf_train.shape, tfidf_test.shape)
print (word2vec_train.shape, word2vec_test.shape)
#print (bert_train.shape, bert_test.shape)

(170004, 14587) (42502, 14587)
(170004, 300) (42502, 300)


### 4. 训练模型以及评估
对如上三种不同的向量表示法，分别训练逻辑回归模型，需要做：
- 搭建模型
- 训练模型（并做交叉验证）
- 输出最好的结果

In [30]:
# 导入逻辑回归的包
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

#### 任务9：使用tf-idf，并结合逻辑回归训练模型

In [31]:
# TODO9: 使用tf-idf + 逻辑回归训练模型，需要用gridsearchCV做交叉验证，并选择最好的超参数

param_grid = {'C': np.arange(0.01, 0.1, 1)}
fit_model = LogisticRegression()
logreg = GridSearchCV(fit_model, param_grid, cv=10)
logreg.fit(tfidf_train, y_train)
print("tuned hpyerparameters :(best parameters) ", logreg.best_params_)
print("accuracy :", logreg.best_score_)

tuned hpyerparameters :(best parameters)  {'C': 0.01}
accuracy : 0.829580476234962


In [32]:
tfidflog = LogisticRegression(C = 0.01)
tfidflog.fit(tfidf_train, y_train)
tf_idf_y_pred = tfidflog.predict(tfidf_test)

print('TF-IDF LR test accuracy %s' % metrics.accuracy_score(y_test, tf_idf_y_pred))
#逻辑回归模型在测试集上的F1_Score
print('TF-IDF LR test F1_score %s' % metrics.f1_score(y_test, tf_idf_y_pred,average="macro"))

TF-IDF LR test accuracy 0.8305726789327561
TF-IDF LR test F1_score 0.4748055092330618


#### 任务10：使用word2vec，并结合逻辑回归训练模型

In [33]:
param_grid = {'C': np.arange(0.01, 0.1,1)}
fit_model = LogisticRegression()
word2vec_log = GridSearchCV(fit_model, param_grid, cv=10)
word2vec_log.fit(word2vec_train, y_train)

print("tuned hpyerparameters :(best parameters) ", word2vec_log.best_params_)
print("accuracy :", word2vec_log.best_score_)

tuned hpyerparameters :(best parameters)  {'C': 0.01}
accuracy : 0.8326392319482936


In [34]:
# TODO10: 使用word2vec + 逻辑回归训练模型，需要用gridsearchCV做交叉验证，并选择最好的超参数
word2vec_logreg = LogisticRegression(C = 0.01)
word2vec_logreg.fit(word2vec_train, y_train)
word2vec_y_pred = word2vec_logreg.predict(word2vec_test)


print('Word2vec LR test accuracy %s' % metrics.accuracy_score(y_test, word2vec_y_pred))
#逻辑回归模型在测试集上的F1_Score
print('Word2vec LR test F1_score %s' % metrics.f1_score(y_test, word2vec_y_pred,average="macro"))

Word2vec LR test accuracy 0.8334196037833513
Word2vec LR test F1_score 0.5104312765711536


#### 任务11：使用bert，并结合逻辑回归训练模型

In [37]:
# TODO11: 使用bert + 逻辑回归训练模型，需要用gridsearchCV做交叉验证，并选择最好的超参数
#param_grid = {'C': np.arange(0.01, 0.1,1)}
#fit_model = LogisticRegression()
#bert_log = GridSearchCV(fit_model, param_grid, cv=10)
#bert_log.fit(word2vec_train, y_train)


#print('Bert LR test accuracy %s' % metrics.accuracy_score(y_test, bert_y_pred))
#逻辑回归模型在测试集上的F1_Score
#print('Bert LR test F1_score %s' % metrics.f1_score(y_test, bert_y_pred,average="macro"))

#### 任务12：对于以上结果请做一下简单的总结，按照1，2，3，4提取几个关键点，包括：
- 结果说明什么问题？
- 接下来如何提高？

1.
2.
3.
4.
5.
6.

In [38]:
# 预处理后的tf-idf和bert生成向量模型的准确率都是在83%左右，但macroF1-score只在50%左右。有可能是样本分布不均导致模型F1较低。
#另外，有可能相同的词汇多种语义会对y产生不同的影响。
#除此之外，我们也可以应用Navie Bayes或者LDA等其他模型来预测评论等级来提高模型。